In [1]:
import os 

In [2]:
%pwd

'd:\\FYP caption Ver 1\\FYP-image-captioning-ver1-transfromers-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd 

'd:\\FYP caption Ver 1\\FYP-image-captioning-ver1-transfromers-'

In [14]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class Data_transformation_config:
    reading_data: Path
    saving_data_train : Path
    saving_data_test : Path
    dataset_processor: str 
    

In [6]:
from imagecaption.constants import *
from imagecaption.utils.common import read_yaml, create_directories
from datasets import load_from_disk

d:\FYP caption Ver 1\FYP-image-captioning-ver1-transfromers-\FYP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-28 15:33:27,103: INFO: config:PyTorch version 2.3.0 available.]


In [59]:

class ConfigurationManger:
    def __init__(self,config_file_path=config_file_path,params_file_path=params_file_path):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])


    def get_data_transformation(self)->Data_transformation_config:
        config= self.config.data_transformation
        
   

        data_transformation_config= Data_transformation_config(
            reading_data=config.reading_data,
            saving_data_train=config.saving_data_train,
            saving_data_test=config.saving_data_test,

            dataset_processor=self.params.TrainingArguments.model_name



        )

        return data_transformation_config

In [54]:
import os 
from imagecaption.logging import logger 
from transformers import AutoProcessor
from torch.utils.data import Dataset, DataLoader

In [55]:
import pickle
from io import open as io_open

In [62]:
class ImageCaptionDataset(Dataset):
           def __init__(self,dataset,processor):
            self.dataset=dataset
            self.processor=processor
           def __len__(self):
             return len(self.dataset)

           def __getitem__(self, idx):
                item = self.dataset[idx]
                encoding = self.processor(images=item["image"], text=item["prompt"], padding="max_length", return_tensors="pt")
                # Remove batch dimension
                encoding = {k: v.squeeze() for k, v in encoding.items()}
                return encoding

In [67]:
class DataTransfromation():
    def __init__(self, config:Data_transformation_config):
         self.config=config 
    
    def datatransfromation(self):
       
       processor=AutoProcessor.from_pretrained(self.config.dataset_processor)
       dataset=load_from_disk(self.config.reading_data)
       
       

       train_dataset=ImageCaptionDataset(dataset['train'],self.config.dataset_processor) 

       valid_dataset = ImageCaptionDataset(dataset['test'], self.config.dataset_processor)
       
       with io_open(self.config.saving_data_train, 'wb') as  file:
          pickle.dump(train_dataset,file) 

       with io_open(self.config.saving_data_test, 'wb') as  file:
          pickle.dump(valid_dataset,file) 


        
       
       
         
         

          
        
        

In [68]:
try:
    config= ConfigurationManger()
    data_transfromation_config=config.get_data_transformation()
    data_transfromation= DataTransfromation(config=data_transfromation_config)
    data_transfromation.datatransfromation()

except Exception as e:
    raise e

[2024-05-28 16:32:26,742: INFO: common:yaml file : config\config.yaml laoded sucessfully]
[2024-05-28 16:32:26,745: INFO: common:yaml file : params.yaml laoded sucessfully]
[2024-05-28 16:32:26,747: INFO: common:created directory at :artifacts ]
